# 4. Truncation without training on 1000 samples t5-small

<hr>

## 1. Load Dataset

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
import json

from datasets import load_dataset
from datasets import load_metric

from rouge_score import rouge_scorer
from transformers import T5Tokenizer, T5ForConditionalGeneration

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
with open('../datasets/test_set.txt') as json_file:
    test_set = json.load(json_file)

In [3]:
data_length = len(test_set['document'])

## 2. Scorers

In [4]:
def compute_metrics(predictions, actuals, tokenizer):
    
    # <your code here>
    metric = load_metric("rouge")
    result = metric.compute(predictions=predictions, references=actuals, use_stemmer=True)
    
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    rouge = {k: round(v, 4) for k, v in result.items()}

    return rouge

## 3. Model

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL = "gniemiec/t5-small-finetuned-xsum"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

print(f"Loaded {MODEL}")

Loaded gniemiec/t5-small-finetuned-xsum


## 4. Evaluation Test

In [6]:
max_length = 512
device = 'cuda:3'

In [7]:
model.eval()

with torch.no_grad():
    # tokens = tokenizer(test_set['document'][1], return_tensors="pt")
    tokens = tokenizer.batch_encode_plus([test_set['document'][1]], max_length=max_length, truncation=True, padding="max_length", return_tensors="pt").to(device)
    # outputs = model.generate(tokens)
    outputs = model.to(device).generate(
                      input_ids = tokens.input_ids,
                      attention_mask = tokens.attention_mask, 
                      max_length=150, 
                      num_beams=2,
                      repetition_penalty=2.5, 
                      length_penalty=1.0, 
                      early_stopping=True
                      )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(decoded)

, all 18, and a woman who has been charged with hate crimes, have appeared in court on Friday.


In [8]:
rouge = compute_metrics([decoded], [test_set['summary'][0]], tokenizer)   
rouge_df = pd.DataFrame.from_dict(rouge, orient='index')
rouge_df

,0
rouge1,34.2857
rouge2,12.1212
rougeL,28.5714
rougeLsum,28.5714
gen_len,1.0000


## 5. Evaluate on 1000 samples

In [9]:
model.eval()
model = model.to(device)
predictions = []

with torch.no_grad():
    for idx, doc in enumerate(test_set['document']):
        if idx % 200 == 0:
            print(f"Completed Epoch {idx} | {data_length}")
        tokens = tokenizer.batch_encode_plus([doc], 
                                             max_length=max_length, 
                                             truncation=True, 
                                             padding="max_length", 
                                             return_tensors="pt").to(device)
        outputs = model.generate(
                      input_ids = tokens.input_ids,
                      attention_mask = tokens.attention_mask, 
                      max_length=150, 
                      num_beams=2,
                      repetition_penalty=2.5, 
                      length_penalty=1.0, 
                      early_stopping=True
                      )
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(decoded)


Completed Epoch 0 | 1000
Completed Epoch 200 | 1000
Completed Epoch 400 | 1000
Completed Epoch 600 | 1000
Completed Epoch 800 | 1000


### 5.1 Saving results

In [10]:
def save_results(predictions, actuals):
    df = pd.DataFrame({'predictions': predictions, 'actuals': actuals})
    df.to_csv('outputs/predictions_pretrained_t5xsum.csv')
    print("PREDICTIONS RESULTS SAVED.")

In [11]:
save_results(predictions, test_set['summary'])

PREDICTIONS RESULTS SAVED.
